In [255]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from efficient_kan import KAN
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [256]:
import random
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(42)

class SoundMaker:
    def __init__(self, head_radius, speed_of_sound=300, min_distance=5, max_distance=500):
        self.head_radius = head_radius
        self.speed_of_sound = speed_of_sound
        self.min_distance = min_distance
        self.max_distance = max_distance

    def make_sound(self):
        azimuth = random.uniform(0, 2 * math.pi)
        distance = random.uniform(self.min_distance, self.max_distance)
        y = distance * math.cos(azimuth)
        x = math.sqrt(distance**2 - y**2) * (-1)**(azimuth > math.pi)
        distance_left = math.sqrt(y**2+(x-self.head_radius)**2)
        distance_right = math.sqrt(y**2+(x+self.head_radius)**2)
        dt = (distance_left - distance_right)/self.speed_of_sound
        return dt, distance, azimuth

soundMaker = SoundMaker(0.1)

In [257]:
n = 10000
df = pd.DataFrame([soundMaker.make_sound() for i in range(n)],columns=["dt","distance","azimuth"])
df

,dt,distance,azimuth
0,0.000512,17.380324,4.017637
1,-0.000658,115.489315,1.728060
2,0.000664,339.966246,4.627385
3,0.000418,48.034722,5.605730
4,-0.000314,19.749624,2.651013
...,...,...,...
9995,0.000530,217.386830,5.364297
9996,-0.000666,9.373110,1.617118
9997,-0.000623,329.837623,1.205442
9998,-0.000600,227.864762,2.021106


In [258]:
input_scaler = StandardScaler()
target_scaler = StandardScaler()
inputs = input_scaler.fit_transform(df[['dt']])
targets = target_scaler.fit_transform(df[['azimuth']])

In [259]:
train_input, test_input, train_target, test_target = train_test_split(inputs, targets, test_size=0.2, random_state=42)

In [260]:
class CustomDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        inputs = torch.tensor(self.inputs[idx], dtype=torch.float32)
        targets = torch.tensor(self.targets[idx], dtype=torch.float32)
        return inputs, targets

In [261]:
# Load your custom dataset
train_dataset = CustomDataset(train_input, train_target)
test_dataset = CustomDataset(test_input, test_target)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [1]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.hidden1 = nn.Linear(input_size, hidden_size)
        self.hidden2 = nn.Linear(hidden_size, output_size)

        nn.init.normal_(self.hidden1.weight, mean=0, std=0.01)
        nn.init.normal_(self.hidden1.bias, mean=0, std=0.01)
        nn.init.normal_(self.hidden2.weight, mean=0, std=0.01)
        nn.init.normal_(self.hidden2.bias, mean=0, std=0.01)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = self.hidden2(x)
        return x

NameError: name 'nn' is not defined

In [ ]:
# For regression
criterion = nn.MSELoss()

In [ ]:
mlp = MLP(input_size=1, hidden_size=4, output_size=1)
kan = KAN([1, 4, 1])

In [ ]:
def train(model, title="MLP", epochs=5):
    optimizer = optim.SGD(mlp.parameters(), lr=1e-3, weight_decay=1e-4)
    # optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    # Define learning rate scheduler
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.8)
    train_loss_all = []
    val_loss_all = []
    losses = []
    for epoch in range(epochs):
        # Train
        train_loss = 0
        train_num = 0
        model.train()
        with tqdm(train_loader) as pbar:
            running_loss = 0.0
            for input, target in pbar:
                input = input.view(-1, 1).to(device)
                optimizer.zero_grad()
                output = model(input)
                loss = criterion(output, target.to(device))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                train_loss += loss.item()*input.size(0)
                train_num += input.size(0)
                pbar.set_postfix(lr=optimizer.param_groups[0]['lr'])
        print('Epoch %d, Train loss: %.3f' % (epoch + 1, train_loss/train_num))
        train_loss_all.append(train_loss/train_num)
        pbar.set_postfix(loss=train_loss/train_num)
    
        # Validation
        mlp.eval()
        val_loss = 0
        val_accuracy = 0
        val_num = 0
        with torch.no_grad():
            for input, target in test_loader:
                input = input.view(-1, 1).to(device)
                output = model(input)
                val_loss += criterion(output, target.to(device)).item()*input.size(0)
                val_num += input.size(0)
        val_loss_all.append(val_loss/val_num)
        # val_accuracy /= len(valloader)
    
        # Update learning rate
        scheduler.step()
    
        print(
            f"Epoch {epoch + 1}, Val Loss: {val_loss/val_num}"
        )
    # Plot the loss values against the number of epochs
    fig, ax = plt.subplots()
    ax.plot(range(1, epoch + 2), train_loss_all, label='Train Loss')
    ax.plot(range(1, epoch + 2), val_loss_all, label='Val Loss')
    ax.set_title(f'Loss Curves {title}')
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.legend()
    plt.show()

In [ ]:
train(mlp, "MLP")

In [2]:
train(kan, "KAN", 10)

NameError: name 'train' is not defined

In [3]:
# Save the trained model
torch.save(mlp.state_dict(), "./models/mlp.pth")
torch.save(kan.state_dict(), "./models/kan.pth")

NameError: name 'torch' is not defined

In [269]:
with torch.no_grad():
    inputs = [0.01, -0.01]
    inputs = np.array([[x] for x in inputs])
    inputs = input_scaler.transform(inputs)
    inputs = torch.tensor(np.array(inputs),dtype=torch.float32)
    print(inputs)
    
    preds = kan(inputs)
    print(preds)
    print(target_scaler.inverse_transform(preds))

tensor([[ 21.1982],
        [-21.2094]])
tensor([[ 3.1310],
        [-0.0041]])
[[8.8394231 ]
 [3.14755325]]


/Users/jeqcho/miniconda3/envs/kanenv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
